In [104]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

In [105]:
# Đọc file CSV
file_path = "movies_data_processed.csv"
df = pd.read_csv(file_path)

# Kiểm tra dữ liệu
df.head()

,name,genres,type,rating,no_of_votes,meta_score,release_date,gross,budget,countries,url,ROI,result
0,Attack on Titan,"Action Epic, Adult Animation, Anime, Shōnen, S...",TV Series,9.1,591000.0,NaN,2013.0,NaN,NaN,Japan,https://www.imdb.com/title/tt2560140/?ref_=sr_...,NaN,Success
1,Love Hurts,"Martial Arts, Action, Comedy",Movie,5.3,6800.0,34.0,2025.0,17235040.0,18000000.0,United States,https://www.imdb.com/title/tt30788842/?ref_=sr...,-0.042498,Fail
2,The Quick and the Dead,"Action, Drama, Romance, Thriller, Western",Movie,6.5,107000.0,49.0,1995.0,18636537.0,32000000.0,"United States, Japan",https://www.imdb.com/title/tt0114214/?ref_=sr_...,-0.417608,Success
3,The Equalizer,"Cop Drama, Police Procedural, Action, Crime, D...",TV Series,5.6,19000.0,NaN,2021.0,NaN,NaN,United States,https://www.imdb.com/title/tt11242246/?ref_=sr...,NaN,Fail
4,Goosebumps,"Teen Horror, Action, Adventure, Comedy, Drama,...",TV Series,6.7,15000.0,NaN,2023.0,NaN,NaN,"United States, Canada, France",https://www.imdb.com/title/tt17720272/?ref_=sr...,NaN,Fail


In [106]:
df.isnull().sum()

name               0
genres             0
type               0
rating           244
no_of_votes      244
meta_score      4348
release_date      58
gross           4461
budget          4887
countries         36
url                0
ROI             5140
result             0
dtype: int64

In [107]:
#dropna cột countries,rating, release_date, no_of_votes
df.dropna(subset=['countries', 'rating', 'release_date', 'no_of_votes'], inplace=True)
df.isnull().sum()

name               0
genres             0
type               0
rating             0
no_of_votes        0
meta_score      4049
release_date       0
gross           4150
budget          4603
countries          0
url                0
ROI             4828
result             0
dtype: int64

In [108]:
# Kiểm tra null trong cột 'gross'
df_Na = df[df.isnull().any(axis=1)]
df_Na.__len__()

df_TV = df_Na[df_Na['type'] == 'TV Series']
print(df_TV.isnull().sum())
df_tv = df[df['type'] == 'TV Series']
print("\n TV Series:")
print(df_tv.describe())

df_movies = df_Na[df_Na['type'] == 'Movie']
print("\n Movies:")
print(df_movies.describe())

# Hiển thị thông tin cơ bản
print(f"Số lượng mẫu có gross là null: {len(df_Na)}")
print(f"Tỷ lệ mẫu null trên tổng số: {len(df_Na)/len(df)*100:.2f}%")

print(f"\nSố lượng mẫu có gross là null và là TV: {len(df_TV)}")
print(f"Tỷ lệ mẫu null trên tổng số và là TV: {len(df_TV)/len(df_Na)*100:.2f}%")

name               0
genres             0
type               0
rating             0
no_of_votes        0
meta_score      3407
release_date       0
gross           3407
budget          3407
countries          0
url                0
ROI             3407
result             0
dtype: int64

 TV Series:
            rating   no_of_votes  meta_score  release_date  gross  budget  ROI
count  3407.000000  3.407000e+03         0.0   3407.000000    0.0     0.0  0.0
mean      7.365101  8.131378e+04         NaN   2010.945700    NaN     NaN  NaN
std       1.269203  1.837625e+05         NaN     14.204456    NaN     NaN  NaN
min       1.500000  6.000000e+00         NaN   1947.000000    NaN     NaN  NaN
25%       6.800000  1.300000e+03         NaN   2006.000000    NaN     NaN  NaN
50%       7.700000  1.100000e+04         NaN   2015.000000    NaN     NaN  NaN
75%       8.300000  8.000000e+04         NaN   2021.000000    NaN     NaN  NaN
max       9.600000  2.400000e+06         NaN   2025.000000    NaN    

- Nhận thấy phần lớn doanh thu, ngân sách, metascore Nan đến từ TV series

In [109]:
df.loc[df['type'] == 'TV Series', ['gross', 'budget','meta_score','ROI']] = df.loc[df['type'] == 'TV Series', ['gross', 'budget','meta_score','ROI']].fillna(0)
df.isnull().sum()

name               0
genres             0
type               0
rating             0
no_of_votes        0
meta_score       642
release_date       0
gross            743
budget          1196
countries          0
url                0
ROI             1421
result             0
dtype: int64

- Nhận thấy các đặc trưng bị NaN có sự phụ thuộc nhau, các phim bị thiếu thường là phim ít được quan tâm, ít đc nhà phê bình đánh giá. Vì vậy, có thể đoán đc doanh thu phim sẽ ko cao.

In [110]:
# Xử lý giá trị thiếu trong các cột số
# Meta_score: điền bằng giá trị ngẫu nhiên từ phân phối chuẩn
if 'meta_score' in df.columns and df['meta_score'].isnull().sum() > 0:
    # Tính tham số phân phối cho meta_score
    meta_mean = df['meta_score'].mean()
    meta_std = df['meta_score'].std()
    
    # Đếm số lượng giá trị null
    null_count = df['meta_score'].isnull().sum()
    
    # Tạo các giá trị ngẫu nhiên từ phân phối chuẩn
    random_values = np.random.normal(loc=meta_mean, scale=meta_std, size=null_count)
    
    # Đảm bảo giá trị nằm trong khoảng hợp lý (meta_score thường từ 0-100)
    random_values = np.clip(random_values, 0, 100)
    
    # Lấy chỉ số của các giá trị null
    null_indices = df['meta_score'].isnull()
    
    # Gán giá trị ngẫu nhiên vào vị trí null
    df.loc[null_indices, 'meta_score'] = pd.Series(random_values, index=df.index[null_indices])
    
    print(f"Cột meta_score: Đã điền {null_count} giá trị NaN bằng phân phối chuẩn với mean={meta_mean:.2f}, std={meta_std:.2f}")

# Các cột khác: điền bằng giá trị thấp nhất khác 0
for column in ['gross', 'budget', 'ROI']:
    # Tìm giá trị thấp nhất khác 0 (bỏ qua giá trị NaN và 0)
    non_zero_min = df[df[column] > 0][column].min()
    
    # Nếu không tìm thấy giá trị dương nào, sử dụng giá trị thấp nhất bất kỳ
    if pd.isna(non_zero_min):
        min_value = df[column].min()
        print(f"Cột {column}: Không tìm thấy giá trị dương, sử dụng giá trị thấp nhất {min_value:.2f}")
    else:
        min_value = non_zero_min
    
    # Đếm số lượng giá trị null
    null_count = df[column].isnull().sum()

    # Điền giá trị NaN bằng giá trị thấp nhất khác 0
    df[column] = df[column].fillna(min_value)
    
    # In thông tin
    print(f"Cột {column}: Đã điền {null_count} giá trị NaN bằng giá trị thấp nhất khác 0: {min_value:.2f}")
    
print("\nKiểm tra sau khi điền:")
print(df.head())
print("\nSố lượng giá trị thiếu còn lại:")
print(df.isnull().sum())

Cột meta_score: Đã điền 642 giá trị NaN bằng phân phối chuẩn với mean=33.91, std=34.82
Cột gross: Đã điền 743 giá trị NaN bằng giá trị thấp nhất khác 0: 19.00
Cột budget: Đã điền 1196 giá trị NaN bằng giá trị thấp nhất khác 0: 10000.00
Cột ROI: Đã điền 1421 giá trị NaN bằng giá trị thấp nhất khác 0: 0.00

Kiểm tra sau khi điền:
                     name                                             genres  \
0         Attack on Titan  Action Epic, Adult Animation, Anime, Shōnen, S...   
1              Love Hurts                       Martial Arts, Action, Comedy   
2  The Quick and the Dead          Action, Drama, Romance, Thriller, Western   
3           The Equalizer  Cop Drama, Police Procedural, Action, Crime, D...   
4              Goosebumps  Teen Horror, Action, Adventure, Comedy, Drama,...   

        type  rating  no_of_votes  meta_score  release_date       gross  \
0  TV Series     9.1     591000.0         0.0        2013.0         0.0   
1      Movie     5.3       6800.0      

In [111]:
# Bỏ các cột ko cần thiết
df = df.drop(columns=['name', 'url', 'result','ROI'])

print("Unique genres:", df['genres'].unique())
print("\nUnique countries:", df['countries'].unique())

Unique genres: ['Action Epic, Adult Animation, Anime, Shōnen, Survival, Tragedy, Action, Adventure, Animation, Drama'
 'Martial Arts, Action, Comedy'
 'Action, Drama, Romance, Thriller, Western' ...
 'Buddy Comedy, Martial Arts, Action, Adventure, Comedy, Western'
 'Classical Western, Costume Drama, Drama, Romance, Western'
 'Animal Adventure, Hand-Drawn Animation, Adventure, Animation, Drama, Family, Western']

Unique countries: ['Japan' 'United States' 'United States, Japan'
 'United States, Canada, France' 'United States, United Kingdom'
 'United States, Canada, Hungary' 'France, United Kingdom'
 'United Kingdom' 'United States, United Kingdom, Switzerland, Panama'
 'India' 'United States, Canada' 'United States, China, Malta'
 'Germany, Canada, United States' 'United States, Australia'
 'Ireland, Canada' 'United States, United Kingdom, Turkey'
 'China, United States' 'South Korea, Czech Republic'
 'United States, United Kingdom, Canada'
 'United Kingdom, United States, South Africa

In [112]:
# Chuyển genres và countries thành danh sách (nếu chứa nhiều giá trị)
df['genres'] = df['genres'].str.split(',')
df['countries'] = df['countries'].str.split(',')

### One-Hot Encoding cho "genres" và "countries"

In [113]:
# Mã hóa genres với index được bảo toàn
mlb_genres = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(
    mlb_genres.fit_transform(df['genres']), 
    columns=mlb_genres.classes_,
    index=df.index  # Quan trọng: giữ nguyên index
)

# Mã hóa countries với index được bảo toàn
mlb_countries = MultiLabelBinarizer()
countries_encoded = pd.DataFrame(
    mlb_countries.fit_transform(df['countries']), 
    columns=mlb_countries.classes_,
    index=df.index  # Quan trọng: giữ nguyên index
)

# Loại bỏ các cột gốc và thêm các cột mã hóa
df_without_encoded = df.drop(columns=['genres', 'countries'])
df = pd.concat([df_without_encoded, genres_encoded, countries_encoded], axis=1)

# Kiểm tra kết quả
df.head()

,type,rating,no_of_votes,meta_score,release_date,gross,budget,Action,Adult Animation,Adventure,...,Sweden,Switzerland,Taiwan,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,West Germany,Yugoslavia
0,TV Series,9.1,591000.0,0.0,2013.0,0.0,0.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Movie,5.3,6800.0,34.0,2025.0,17235040.0,18000000.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Movie,6.5,107000.0,49.0,1995.0,18636537.0,32000000.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,TV Series,5.6,19000.0,0.0,2021.0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,TV Series,6.7,15000.0,0.0,2023.0,0.0,0.0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [114]:
df.isnull().sum()

type                    0
rating                  0
no_of_votes             0
meta_score              0
release_date            0
                       ..
United Arab Emirates    0
United Kingdom          0
United States           0
West Germany            0
Yugoslavia              0
Length: 549, dtype: int64

### Label Encoding cho "type"

In [115]:
label_encoder_type = LabelEncoder()
df['type'] = label_encoder_type.fit_transform(df['type'])
df.head()

,type,rating,no_of_votes,meta_score,release_date,gross,budget,Action,Adult Animation,Adventure,...,Sweden,Switzerland,Taiwan,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,West Germany,Yugoslavia
0,1,9.1,591000.0,0.0,2013.0,0.0,0.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,5.3,6800.0,34.0,2025.0,17235040.0,18000000.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,6.5,107000.0,49.0,1995.0,18636537.0,32000000.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,5.6,19000.0,0.0,2021.0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,6.7,15000.0,0.0,2023.0,0.0,0.0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [116]:
df.isnull().sum()

type                    0
rating                  0
no_of_votes             0
meta_score              0
release_date            0
                       ..
United Arab Emirates    0
United Kingdom          0
United States           0
West Germany            0
Yugoslavia              0
Length: 549, dtype: int64

In [117]:
df.to_csv("movies_data_encoded_full.csv", index=False)